In [1]:
%load_ext nb_black
import pandas as pd
import numpy as np
import regex as re

<IPython.core.display.Javascript object>

In [2]:
data = "datasets/challenge_dataset"
immo_eliza = pd.read_table(data, sep=",")
df = immo_eliza[["locality", "postcode", "kitchen_has", "furnished", "facades_number"]]
df.head()

/home/sara/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (11,12,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,locality,postcode,kitchen_has,furnished,facades_number
0,4180,4180.0,True,False,0
1,8730,8730.0,True,False,0
2,4020,4020.0,True,False,0
3,1200,1200.0,True,True,0
4,1190,1190.0,True,False,0


<IPython.core.display.Javascript object>

# Functions

Functions here clean the following columns:

- **postcode**
- **furnished**
- **kitchen_has**
- **facades_number**

Modifications on the **locality** column will be performed once the data of the 4 members of the group is merged.

In [3]:
def postcodes_fun():
    "This function extracts postcodes from locality and adds them into null postcodes"

    # Extract postcodes from locality and add them to postcode column
    df["loc_code"] = df["locality"].str.extract(r"(\b[0-9]{4}\b)")
    df.loc[df.postcode.isnull(), "postcode"] = df.loc_code

    # Fill null vales with '-999'
    df["postcode"].fillna("-999", inplace=True)

    # Delete dummy column
    del df["loc_code"]

    # Return column
    return df["postcode"]


def bool_to_bin(x, final_type):
    """This function converts a True/False column into a 0/1"""

    df[x] = df[x].astype(str)
    df[x] = df[x].map({"False": 0, "True": 1, "nan": -999})
    df[x].fillna(-999, inplace=True)
    df[x] = df[x].astype(final_type)
    return df[x]


def facades_num():
    """This function cleans facades_number by covnerting values into integers
    and assign '0', 'nan' and 'none' to -999 (=treat as null)"""

    df["facades_number"] = df["facades_number"].apply(str)
    df.facades_number.replace("(\.0)", "", regex=True, inplace=True)
    df["facades_number"] = df.facades_number.replace(
        ["nan", "None", "0"], -999
    )  # maybe treat 0 as -999 too?
    df["facades_number"] = df["facades_number"].astype(int)
    return df["facades_number"]

<IPython.core.display.Javascript object>

### Calling functions

In [4]:
df["postcode"] = postcodes_fun()
df["furnished"] = bool_to_bin("furnished", int)
df["kitchen_has"] = bool_to_bin("kitchen_has", int)
df["facades_number"] = facades_num()

/home/sara/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/sara/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/sara/.local/lib/python3.6/site-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

<IPython.core.display.Javascript object>

## Locality: to be executed AFTER merging

As locality values are not accurate in our dataset, we pick (cleaned) postal codes as reference. Now we can assign an accurate locality value to these postal codes by merging the file with an external dataset with all Belgian postcodes. This allows us to also make sure that no postal codes other than Belgian are in our dataset. 

In [5]:
# 1. Assign postcode column as the dataframe index
df.set_index(df["postcode"], inplace=True)
df.index = df.index.astype(int)
df

,locality,postcode,kitchen_has,furnished,facades_number
postcode,,,,,
4180,4180,4180,1,0,-999
8730,8730,8730,1,0,-999
4020,4020,4020,1,0,-999
1200,1200,1200,1,1,-999
1190,1190,1190,1,0,-999
...,...,...,...,...,...
2650,2650,2650,1,-999,2
3700,3700,3700,1,-999,4
3800,3800,3800,1,-999,2


<IPython.core.display.Javascript object>

In [6]:
# 2. Load external dataset of belgian postal codes
data_postcodes = "datasets/be_postal_codes.csv"
loc_pc = pd.read_csv(data_postcodes, encoding="latin")
loc_pc

,Postal Code,Place Name,State,State Abbreviation,City,Latitude,Longitude
0,1000,Bruxelles,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
1,1005,Conseil Region Bruxelles-Capitale,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
2,1006,Raad Vlaamse Gemeenschapscommissie,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
3,1007,Ass. Commiss. Communau. française,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
4,1008,Chambre des Représentants,Bruxelles-Capitale,BRU,Bruxelles,50.8466,4.3528
...,...,...,...,...,...,...,...
1170,7972,Beloeil Ellignies-Sainte-Anne,Wallonie,WAL,Hainaut,50.5618,3.6727
1171,7973,Beloeil Grandglise,Wallonie,WAL,Hainaut,50.5057,3.6963
1172,1100,Postcheque,NaN,NaN,NaN,50.8350,4.3647
1173,1105,SOC,NaN,NaN,NaN,50.8350,4.3647


<IPython.core.display.Javascript object>

In [7]:
# 3. Merge files by matching index of left file with "Postal Code" column of right file
merged_df = df.merge(loc_pc, left_index=True, right_on="Postal Code")
merged_df

,locality,postcode,kitchen_has,furnished,facades_number,Postal Code,Place Name,State,State Abbreviation,City,Latitude,Longitude
637,4180,4180,1,0,-999,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
637,4180,4180,1,0,-999,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
637,4180,4180,0,0,-999,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
637,4180,4180,1,0,-999,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
637,4180,4180,1,0,-999,4180,Hamoir Comblain-Fairon,Wallonie,WAL,Liège,50.4500,5.5500
...,...,...,...,...,...,...,...,...,...,...,...,...
361,8421,8421,1,0,3,8421,De Haan,Vlaanderen,VLG,West-Vlaanderen,51.2667,3.0333
361,8421,8421,1,-999,3,8421,De Haan,Vlaanderen,VLG,West-Vlaanderen,51.2667,3.0333
899,6441,6441,1,0,-999,6441,Froidchapelle Erpion,Wallonie,WAL,Hainaut,50.0574,4.3157
15,1044,1044,-999,-999,4,1044,RTBF,Bruxelles-Capitale,BRU,Bruxelles,50.8412,4.3704


<IPython.core.display.Javascript object>

In [8]:
# 4. Reset index of this new merged file
merged_df = merged_df.reset_index()

<IPython.core.display.Javascript object>

In [9]:
# 5. Create new data file containing only the columns we are interested in (in the right order)
complete_data = merged_df[merged_df.columns[[7, 2, 10, 8, 3, 4, 5]]]
complete_data

,Place Name,postcode,City,State,kitchen_has,furnished,facades_number
0,Hamoir Comblain-Fairon,4180,Liège,Wallonie,1,0,-999
1,Hamoir Comblain-Fairon,4180,Liège,Wallonie,1,0,-999
2,Hamoir Comblain-Fairon,4180,Liège,Wallonie,0,0,-999
3,Hamoir Comblain-Fairon,4180,Liège,Wallonie,1,0,-999
4,Hamoir Comblain-Fairon,4180,Liège,Wallonie,1,0,-999
...,...,...,...,...,...,...,...
87240,De Haan,8421,West-Vlaanderen,Vlaanderen,1,0,3
87241,De Haan,8421,West-Vlaanderen,Vlaanderen,1,-999,3
87242,Froidchapelle Erpion,6441,Hainaut,Wallonie,1,0,-999
87243,RTBF,1044,Bruxelles,Bruxelles-Capitale,-999,-999,4


<IPython.core.display.Javascript object>